In [3]:
import pandas as pd

In [4]:
# function to read txt data files and convert them to proper csv files
# txtFile: input filename (including directory if applicable)
# csvFile: output filename (including directory if applicable)
# vtabchar: vertical tab character in the original file (to be replaced with newline command '\n')
# delim: delimiter character used in the original file (to be replaced with comma)
def txt2csv(txtFile, csvFile, vtabchar, delim):
    with open(txtFile, 'r') as file:
        data = file.read().replace(vtabchar, '\n').replace(delim, ',')
    with open(csvFile, 'w') as file:
        file.write(data)    
    return

Set the filename and location for each dataset

In [5]:
# original filename and directory for txt data files
K1DI2_txt = './Data/Komponente/Komponente_K1DI2.txt'
K2LE1_txt = './Data/Komponente/Komponente_K2LE1.txt' 
K2LE2_txt = './Data/Komponente/Komponente_K2LE2.txt'
K2ST1_txt = './Data/Komponente/Komponente_K2ST1.txt'
K3AG2_txt = './Data/Komponente/Komponente_K3AG2.txt'
K7_txt    = './Data/Komponente/Komponente_K7.txt'

# converted txt filename and directory
K1DI2_csv = './Data/Komponente/Komponente_K1DI2.csv'
K2LE1_csv = './Data/Komponente/Komponente_K2LE1.csv'
K2LE2_csv = './Data/Komponente/Komponente_K2LE2.csv'
K2ST1_csv = './Data/Komponente/Komponente_K2ST1.csv'
K3AG2_csv = './Data/Komponente/Komponente_K3AG2.csv'
K7_csv    = './Data/Komponente/Komponente_K7.csv'

# original filename and directory for csv data files
# component data files
K1BE1_csv = './Data/Komponente/Komponente_K1BE1.csv'
K1BE2_csv = './Data/Komponente/Komponente_K1BE2.csv'
K1DI1_csv = './Data/Komponente/Komponente_K1DI1.csv'
K2ST2_csv = './Data/Komponente/Komponente_K2ST2.csv'
K3AG1_csv = './Data/Komponente/Komponente_K3AG1.csv'
K3SG1_csv = './Data/Komponente/Komponente_K3SG1.csv'
K3SG2_csv = './Data/Komponente/Komponente_K3SG2.csv'
K4_csv    = './Data/Komponente/Komponente_K4.csv'
K5_csv    = './Data/Komponente/Komponente_K5.csv'
K6_csv    = './Data/Komponente/Komponente_K6.csv'    

# vehicle data files
bestFahr1_11_csv = './Data/Fahrzeug/Bestandteile_Fahrzeuge_OEM1_Typ11.csv'
bestFahr1_12_csv = './Data/Fahrzeug/Bestandteile_Fahrzeuge_OEM1_Typ12.csv'
bestFahr2_21_csv = './Data/Fahrzeug/Bestandteile_Fahrzeuge_OEM2_Typ21.csv'
bestFahr2_22_csv = './Data/Fahrzeug/Bestandteile_Fahrzeuge_OEM2_Typ22.csv'

fahr1_11_csv = './Data/Fahrzeug/Fahrzeuge_OEM1_Typ11.csv'
fahr1_12_csv = './Data/Fahrzeug/Fahrzeuge_OEM1_Typ12.csv'
fahr2_21_csv = './Data/Fahrzeug/Fahrzeuge_OEM2_Typ21.csv'
fahr2_22_csv = './Data/Fahrzeug/Fahrzeuge_OEM2_Typ22.csv'

In [6]:
# read and convert all the txt data files to csv
txt2csv(K1DI2_txt, K1DI2_csv, '	', '\\')
txt2csv(K2LE1_txt, K2LE1_csv, '', 'II')
txt2csv(K2LE2_txt, K2LE2_csv, '', '\\')
txt2csv(K2ST1_txt, K2ST1_csv, '', '|')
txt2csv(K3AG2_txt, K3AG2_csv, '', '\\')
txt2csv(K7_txt   , K7_csv   , '', '	')

In [7]:
# separate the data arrangements into 4 types, namely A, B, C, and D. 
# this separation is based on the column names of the datetime data type
A = ['Fehlerhaft_Datum', 'origin']
B = ['Produktionsdatum.x', 'Fehlerhaft_Datum.x', 
     'Produktionsdatum.y', 'Fehlerhaft_Datum.y']
C = ['Produktionsdatum.x', 'Fehlerhaft_Datum.x', 
     'Produktionsdatum.y', 'Fehlerhaft_Datum.y', 
     'Produktionsdatum', 'Fehlerhaft_Datum']
D = ['Produktionsdatum', 'Fehlerhaft_Datum']

# set up a function to read the csv files
def csvReader(csvFile, arr_type, delim=None):
    if delim is not None:
        dataset = pd.read_csv(csvFile, parse_dates=arr_type, 
                          low_memory=False, sep=delim)
    else:
        dataset = pd.read_csv(csvFile, parse_dates=arr_type, 
                          low_memory=False)
    return dataset

# read the converted csv files using the csvReader function
K1DI2 = csvReader(K1DI2_csv, A)
K2LE1 = csvReader(K2LE1_csv, B)
K2LE2 = csvReader(K2LE2_csv, A)
K2ST1 = csvReader(K2ST1_csv, D)
K3AG2 = csvReader(K3AG2_csv, A)
K7    = csvReader(K7_csv, A)

# read the rest of the csv files using the csvReader function
K1BE1 = csvReader(K1BE1_csv, A)
K1BE2 = csvReader(K1BE2_csv, A, ';')
K1DI1 = csvReader(K1DI1_csv, C)
K2ST2 = csvReader(K2ST2_csv, A, ';')
K3AG1 = csvReader(K3AG1_csv, C)
K3SG1 = csvReader(K3SG1_csv, B)
K3SG2 = csvReader(K3SG2_csv, A)
K4    = csvReader(K4_csv,    B, ';')
K5    = csvReader(K5_csv,    B)
K6    = csvReader(K6_csv,    A, ';')

# read the vehicle data:
bestFahr1_11 = pd.read_csv(bestFahr1_11_csv)
bestFahr1_12 = pd.read_csv(bestFahr1_12_csv)
bestFahr2_21 = pd.read_csv(bestFahr2_21_csv, sep=';')
bestFahr2_22 = pd.read_csv(bestFahr2_22_csv, sep=';')
# fahr1_11 = pd.read_csv(fahr1_11_csv, parse_dates=D, low_memory=False)
# fahr1_12 = pd.read_csv(fahr1_12_csv, parse_dates=D, low_memory=False)
# fahr2_21 = pd.read_csv(fahr2_21_csv, parse_dates=A, low_memory=False)
# fahr2_22 = pd.read_csv(fahr2_22_csv, parse_dates=A, low_memory=False, sep=';')

For datasets with data arrangements of type B and C, we need to consolidate the columns and eliminate the .x and .y suffixes. For type B, the tables are separated into 2, whereas for type C, the tables are separated into 3.

In [8]:
# column names to be renamed for type B
col_names_x = {'Produktionsdatum.x':'Produktionsdatum', 
               'Herstellernummer.x':'Herstellernummer',	
               'Werksnummer.x':'Werksnummer',
               'Fehlerhaft.x':'Fehlerhaft', 
               'Fehlerhaft_Datum.x':'Fehlerhaft_Datum',
               'Fehlerhaft_Fahrleistung.x':'Fehlerhaft_Fahrleistung'}
col_names_y = {'Produktionsdatum.y':'Produktionsdatum', 
               'Herstellernummer.y':'Herstellernummer',	
               'Werksnummer.y':'Werksnummer',
               'Fehlerhaft.y':'Fehlerhaft', 
               'Fehlerhaft_Datum.y':'Fehlerhaft_Datum',
               'Fehlerhaft_Fahrleistung.y':'Fehlerhaft_Fahrleistung'}

# set up variables for the components. The components are engine (Motor), electrical components (Schaltung),
# body components (Karosserie), and seats (Sitze)
idMotor  = 'ID_Motor'        ; idSchalt  = 'ID_Schaltung'
idMotorx = 'ID_Motor.x'      ; idSchaltx = 'ID_Schaltung.x'
idMotory = 'ID_Motor.y'      ; idSchalty = 'ID_Schaltung.y'

idKaros  = 'ID_Karosserie'   ; idSitze  = 'ID_Sitze'
idKarosx = 'ID_Karosserie.x' ; idSitzex = 'ID_Sitze.x' 
idKarosy = 'ID_Karosserie.y' ; idSitzey = 'ID_Sitze.y'

# separate type B tables based on the suffixes, rename the columns, and 
# concatenate vertically, and finally extract the columns that contain the data.
# the cleaned up dataset is added with suffix '_c'
def streamlineTypeB(dataset, colx, coly, ID, IDx, IDy):
    dataset_x = dataset[dataset[IDx].notna()].rename(columns=colx)
    dataset_y = dataset[dataset[IDy].notna()].rename(columns=coly)
    dataset_x = dataset_x.rename(columns={IDx : ID})
    dataset_y = dataset_y.rename(columns={IDy : ID})
    dataset_x = dataset_x.loc[:, ID:'Fehlerhaft_Fahrleistung']
    dataset_y = dataset_y.loc[:, ID:'Fehlerhaft_Fahrleistung']
    dataset_c = pd.concat([dataset_x, dataset_y], axis=0).reset_index(drop=True)
    dataset_c['Fehlerhaft'] = dataset_c['Fehlerhaft'].astype('bool')
    return dataset_c

K2LE1_c = streamlineTypeB(K2LE1, col_names_x, col_names_y, 
                            idSitze, idSitzex, idSitzey)
K3SG1_c = streamlineTypeB(K3SG1, col_names_x, col_names_y, 
                            idSchalt, idSchaltx, idSchalty)
K4_c    = streamlineTypeB(K4, col_names_x, col_names_y, 
                            idKaros, idKarosx, idKarosy)
K5_c    = streamlineTypeB(K5, col_names_x, col_names_y, 
                            idKaros, idKarosx, idKarosy)

# separate type C tables based on the suffixes, rename the columns, and 
# concatenate vertically, and finally extract the columns that contain the data.
# the cleaned up dataset is added with suffix '_c'
def streamlineTypeC(dataset, colx, coly, ID, IDx, IDy):
    dataset_x = dataset[dataset[IDx].notna()].loc[:, IDx:'Fehlerhaft_Fahrleistung.x'].rename(columns=colx)
    dataset_y = dataset[dataset[IDy].notna()].loc[:, IDy:'Fehlerhaft_Fahrleistung.y'].rename(columns=coly)
    dataset_  = dataset[dataset[ID].notna()]
    dataset_x = dataset_x.rename(columns={IDx : ID})
    dataset_y = dataset_y.rename(columns={IDy : ID})
    dataset_  = dataset_.loc[:, ID:'Fehlerhaft_Fahrleistung']    
    dataset_c = pd.concat([dataset_x, dataset_y, dataset_], axis=0).reset_index(drop=True)
    dataset_c['Fehlerhaft'] = dataset_c['Fehlerhaft'].astype('bool')
    return dataset_c

K1DI1_c = streamlineTypeC(K1DI1, col_names_x, col_names_y,
                          idMotor, idMotorx, idMotory)
K3AG1_c = streamlineTypeC(K3AG1, col_names_x, col_names_y,
                          idSchalt, idSchaltx, idSchalty)
K2ST1_c = K2ST1.loc[:, idSitze:'Fehlerhaft_Fahrleistung']
K2ST1_c['Fehlerhaft'] = K2ST1_c['Fehlerhaft'].astype('bool')

In [9]:
def addProduktionsDatum(dataset):
    dataset['Produktionsdatum'] = dataset['origin'] + pd.to_timedelta(dataset['Produktionsdatum_Origin_01011970'].astype('int'), unit='days')
    return dataset

def reformatDataset(dataset, ID):
    dataset_c = pd.concat([dataset.loc[:,ID], 
                           dataset.loc[:,'Produktionsdatum'], 
                           dataset.loc[:,'Herstellernummer':'Fehlerhaft_Fahrleistung']], axis=1)
    dataset_c['Fehlerhaft'] = dataset_c['Fehlerhaft'].astype('bool')
    return dataset_c

K1BE1_c = reformatDataset(addProduktionsDatum(K1BE1), idMotor)
K1BE2_c = reformatDataset(addProduktionsDatum(K1BE2), idMotor)
K1DI2_c = reformatDataset(addProduktionsDatum(K1DI2), idMotor)
K2LE2_c = reformatDataset(addProduktionsDatum(K2LE2), idSitze)
K2ST2_c = reformatDataset(addProduktionsDatum(K2ST2), idSitze)
K3AG2_c = reformatDataset(addProduktionsDatum(K3AG2), idSchalt)
K3SG2_c = reformatDataset(addProduktionsDatum(K3SG2), idSchalt)
K6_c    = reformatDataset(addProduktionsDatum(K6), idKaros)
K7_c    = reformatDataset(addProduktionsDatum(K7), idKaros)

# classify the components into list based on the types of components (Motor, Sitze, Schaltung, and Karosserie)
motorList      = [K1BE1_c, K1BE2_c, K1DI1_c, K1DI2_c]
sitzeList      = [K2LE1_c, K2LE2_c, K2ST1_c, K2ST2_c]
schaltungList  = [K3AG1_c, K3AG2_c, K3SG1_c, K3SG2_c]
karosserieList = [K4_c, K5_c, K6_c, K7_c]

motorName      = ['K1BE1', 'K1BE2', 'K1DI1', 'K1DI2']
sitzeName      = ['K2LE1', 'K2LE2', 'K2ST1', 'K2ST2']
schaltungName  = ['K3AG1', 'K3AG2', 'K3SG1', 'K3SG2']
karosserieName = ['K4', 'K5', 'K6', 'K7']

# checks for duplicates in the components datalists and drop them if preset
def dropDuplicates(datasetList):
    for i in range(len(datasetList)):
        datasetList[i] = datasetList[i].drop_duplicates()
        datasetList[i] = datasetList[i].reset_index(drop=True)
    return datasetList

motorList      = dropDuplicates(motorList)
sitzeList      = dropDuplicates(sitzeList)
schaltungList  = dropDuplicates(schaltungList)
karosserieList = dropDuplicates(karosserieList)


Combine all the data of the vehicle brands, types, components installed into a single dataset

In [10]:
# concatenate all the vehicle and components dataset (bestFahr1_11, bestFahr1_12, bestFahr2_21, and bestFahr2_22) into a single dataset
# columns to be used:
col_Fahr_Komponente = ['ID_Karosserie','ID_Schaltung','ID_Sitze','ID_Motor','ID_Fahrzeug']
bestFahr = pd.concat([bestFahr1_11.loc[:, col_Fahr_Komponente],
                      bestFahr1_12.loc[:, col_Fahr_Komponente],
                      bestFahr2_21.loc[:, col_Fahr_Komponente],
                      bestFahr2_22.loc[:, col_Fahr_Komponente]], axis=0).reset_index(drop=True)

Now with all the datasets clean and proper, we can start doing the case analysis. In this case study, the components produced between January 1st, 2009 and December 31st, 2015 are considered. 

First, we filter out the components that is not produced within the considered time period.

In [11]:
# create a function that will extract the dataset within the specified time period
def timePeriodFilter(dataset, ID):
    startDate = '2009-01-01'
    endDate   = '2015-12-31'
    dataset_2009until2015 = pd.concat([dataset[(dataset['Produktionsdatum'] >= startDate) & 
                                       (dataset['Produktionsdatum'] <= endDate)].loc[:,[ID,'Produktionsdatum']],
                                       dataset[(dataset['Produktionsdatum'] >= startDate) & 
                                       (dataset['Produktionsdatum'] <= endDate)].loc[:,'Herstellernummer':'Fehlerhaft_Fahrleistung']],
                                       axis=1).reset_index(drop=True)
    return dataset_2009until2015

# the function below will slice all the components dataset to the specified time range
def component_2009until2015(datasetList, ID):
    datasetlist_2009until2015 = []
    for dataset in datasetList:
        datasetlist_2009until2015.append(timePeriodFilter(dataset, ID))
    return datasetlist_2009until2015

motorList_2009until2015      = component_2009until2015(motorList, idMotor)
sitzeList_2009until2015      = component_2009until2015(sitzeList, idSitze)
schaltungList_2009until2015  = component_2009until2015(schaltungList, idSchalt)
karosserieList_2009until2015 = component_2009until2015(karosserieList, idKaros)

Now, we create the cleaned dataset that contains the components information along with the vehicle that it was installed in. The columns consist component type, part number, date of production, supplier number, plant number, failure status, date of failure, failure mileage, and vehicle number. First, we concatenate all the components of the same type, then concatenate all the components together. Then merge the dataset with the vehicle-components dataset using the components' part number. Save this dataset as the final data in the root folder.

In [12]:
# create a function to concatenate the datasets of each component type, merge the component dataset with the vehicle dataset,
# and rename the ID columns of all the dataset.
def concatenateDataset(datasetList_2009until2015, id):
    for i in (1, len(motorList_2009until2015) - 1):
        datasetList_2009until2015_all = pd.concat([datasetList_2009until2015[0], datasetList_2009until2015[i]], axis=0)
        datasetList_2009until2015[0]  = datasetList_2009until2015_all
    datasetList_2009until2015_all = pd.merge(datasetList_2009until2015_all, bestFahr.loc[:,[id,'ID_Fahrzeug']], on=id)
    datasetList_2009until2015_all = datasetList_2009until2015_all.rename(columns={id:'ID'})
    return datasetList_2009until2015_all

# concatenate the components based on type
motorList_2009until2015_all      = concatenateDataset(motorList_2009until2015, idMotor)
sitzeList_2009until2015_all      = concatenateDataset(sitzeList_2009until2015, idSitze)
schaltungList_2009until2015_all  = concatenateDataset(schaltungList_2009until2015, idSchalt)
karosserieList_2009until2015_all = concatenateDataset(karosserieList_2009until2015, idKaros)

# concatenate all the components into a single dataset
komponente_2009until2015_all = pd.concat([motorList_2009until2015_all,
                                          sitzeList_2009until2015_all,
                                          schaltungList_2009until2015_all,
                                          karosserieList_2009until2015_all], axis=0, join='inner')
komponente_2009until2015_all

# save the cleaned file to the final dataset
komponente_2009until2015_all.to_csv('Final_dataset_group_33.csv')

,ID,Produktionsdatum,Herstellernummer,Werksnummer,Fehlerhaft,Fehlerhaft_Datum,Fehlerhaft_Fahrleistung,ID_Fahrzeug
0,K1BE1-104-1041-11669,2009-01-01,104.0,1041.0,False,NaT,0.0,11-1-11-18713
1,K1BE1-104-1041-11813,2009-01-01,104.0,1041.0,False,NaT,0.0,11-1-11-18739
2,K1BE1-104-1041-11892,2009-01-01,104.0,1041.0,False,NaT,0.0,11-1-11-18880
3,K1BE1-104-1041-11902,2009-01-01,104.0,1041.0,False,NaT,0.0,11-1-11-18962
4,K1BE1-104-1041-11875,2009-01-01,104.0,1041.0,True,2010-05-26,31009.0,11-1-11-18996
...,...,...,...,...,...,...,...,...
2356687,K7-113-1132-120108,2015-12-31,112.0,1132.0,True,NaT,0.0,22-2-22-216142
2356688,K7-113-1132-120127,2015-12-31,112.0,1132.0,True,NaT,0.0,22-2-22-216143
2356689,K7-113-1132-120066,2015-12-31,112.0,1132.0,True,NaT,0.0,22-2-22-216217
2356690,K7-113-1132-120036,2015-12-30,112.0,1132.0,True,NaT,0.0,22-2-21-52130


In [30]:
komponente_2009until2015_all.to_csv('Final_dataset_group_33.csv')

Create a function to calculate the relative frequency of defective components: 

In [13]:
# prepare a function to calculate relative frequency
def calcRelFreq(dataset, ID, productionYear):
    relFreq = []
    for year in productionYear:
        numDefect = dataset[(dataset['Fehlerhaft'] == True) &
                            (dataset['Produktionsdatum'].dt.year == year)].loc[:,'Fehlerhaft'].count()
        numKompon = dataset[ID].count()
        relFreq.append(numDefect/numKompon)
    return relFreq

Prepare lists of components that will be use to store the annual relative frequencies of the components

In [14]:
# set the production year
productionYear = [2009, 2010, 2011, 2012, 2013, 2014, 2015]
productionYear_dict = {'ProductionYear':productionYear}

# create a function that will be used to calculate annual relative frequency
# of components in the components list
def component_relFrq(datasetlist, ID, prodYear):
    datasetlist_relFrq = []
    for dataset in datasetlist:
        datasetlist_relFrq.append(calcRelFreq(dataset, ID, prodYear))
    return datasetlist_relFrq

motorList_relFrq      = component_relFrq(motorList_2009until2015, idMotor, productionYear)
sitzeList_relFrq      = component_relFrq(sitzeList_2009until2015, idSitze, productionYear)
schaltungList_relFrq  = component_relFrq(schaltungList_2009until2015, idSchalt, productionYear)
karosserieList_relFrq = component_relFrq(karosserieList_2009until2015, idKaros, productionYear)

Create dataframes of the relative frequency for all the components produced between January 1st, 2009 and December 31st, 2015

In [15]:
# create a dataframe of the production year
productionYearDF = pd.DataFrame(productionYear_dict)
# create a function to create a list of dataframes of the components' failure 
# relative frequency
def createDataFrame(datasetList_relFrq, componentName):
    relFreqDataframe = []
    for i in range(len(datasetList_relFrq)):
        relFreqDataframe.append(pd.DataFrame(datasetList_relFrq[i], columns=[componentName[i]]))
    return relFreqDataframe

motorList_relFrqDataframe      = createDataFrame(motorList_relFrq, motorName)
sitzeList_relFrqDataframe      = createDataFrame(sitzeList_relFrq, sitzeName)
schaltungList_relFrqDataframe  = createDataFrame(schaltungList_relFrq, schaltungName)
karosserieList_relFrqDataframe = createDataFrame(karosserieList_relFrq, karosserieName)

# combine dataframes in the list into a single dataframe for each component type
def combineDF(datasetList_relFrqDataframe):
    for i in range(1, len(datasetList_relFrqDataframe)):
        datasetListRelFrqDataframe = datasetList_relFrqDataframe[0].join(datasetList_relFrqDataframe[i])
        datasetList_relFrqDataframe[0] = datasetListRelFrqDataframe
    datasetListRelFrqDataframe = pd.concat([productionYearDF, datasetListRelFrqDataframe], axis=1)
    datasetListRelFrqDataframe.set_index('ProductionYear')
    return datasetListRelFrqDataframe

motorListRelFrqDataframe = combineDF(motorList_relFrqDataframe)
sitzeListRelFrqDataframe = combineDF(sitzeList_relFrqDataframe)
schaltungListRelFrqDataframe = combineDF(schaltungList_relFrqDataframe)
karosserieListRelFrqDataframe = combineDF(karosserieList_relFrqDataframe)

In [16]:
motorListRelFrqDataframe

,ProductionYear,K1BE1,K1BE2,K1DI1,K1DI2
0,2009,0.048809,0.024785,0.014287,0.142529
1,2010,0.048852,0.025103,0.014374,0.142636
2,2011,0.049127,0.025011,0.014393,0.142859
3,2012,0.049165,0.025296,0.014420,0.143581
4,2013,0.049202,0.024833,0.014099,0.143082
5,2014,0.049202,0.025262,0.014416,0.143311
6,2015,0.048743,0.024981,0.014350,0.142002


In [17]:
sitzeListRelFrqDataframe

,ProductionYear,K2LE1,K2LE2,K2ST1,K2ST2
0,2009,0.077355,0.139060,0.142828,0.014243
1,2010,0.077528,0.140518,0.142779,0.014295
2,2011,0.077817,0.141789,0.142785,0.014317
3,2012,0.077769,0.142576,0.143191,0.014173
4,2013,0.078099,0.144703,0.142834,0.014338
5,2014,0.078419,0.144406,0.142797,0.014710
6,2015,0.078395,0.146948,0.142786,0.014434


In [18]:
schaltungListRelFrqDataframe

,ProductionYear,K3AG1,K3AG2,K3SG1,K3SG2
0,2009,0.032143,0.142428,0.014161,0.014380
1,2010,0.031926,0.142930,0.014323,0.014163
2,2011,0.032009,0.142554,0.014308,0.014108
3,2012,0.032104,0.142881,0.014147,0.014099
4,2013,0.032312,0.143452,0.014234,0.014452
5,2014,0.031870,0.142018,0.014153,0.014241
6,2015,0.032157,0.143737,0.014171,0.014188


In [19]:
karosserieListRelFrqDataframe

,ProductionYear,K4,K5,K6,K7
0,2009,0.028914,0.014396,0.014193,0.142579
1,2010,0.028888,0.014211,0.013991,0.142832
2,2011,0.028897,0.014337,0.014422,0.143004
3,2012,0.029024,0.014265,0.014320,0.143179
4,2013,0.028919,0.014379,0.014389,0.142944
5,2014,0.028957,0.014659,0.014333,0.142769
6,2015,0.028797,0.014074,0.014264,0.142694
